# 🚀 Day 2-4: 결정 경계 찾기(SVM) & 확률 기반 예측기(Naive Bayes)

지금까지 우리는 데이터를 분리하는 '선'을 찾는 여러 분류 모델을 배웠습니다. 그렇다면 가장 이상적인 '선', 즉 **최적의 결정 경계(Decision Boundary)** 는 어떻게 찾을 수 있을까요? 그리고 완전히 다른 접근법으로, 데이터의 확률적 특성을 활용하여 분류하는 방법은 없을까요?

이번 시간에는 두 가지 강력하고 대조적인 분류 알고리즘을 탐험합니다.

1.  **서포트 벡터 머신 (Support Vector Machine, SVM)**: 클래스 간의 간격(Margin)을 최대화하는 가장 '안정적인' 결정 경계를 찾는 알고리즘입니다. 
   
    '커널 트릭'이라는 마법을 통해 비선형 데이터까지 완벽하게 분리해내는 능력을 보여줍니다.
2.  **나이브 베이즈 (Naive Bayes)**: 베이즈 정리에 기반하여 각 클래스에 속할 '확률'을 계산하는 확률적 분류 모델입니다. 
    
    특성들이 서로 독립적이라는 '순진한(Naive)' 가정을 하지만, 텍스트 분류와 같은 특정 분야에서 놀랍도록 빠르고 효과적인 성능을 자랑합니다.

이번 실습에서는 커널 트릭의 힘을 시각적으로 확인하기 위해 **Fashion-MNIST 데이터셋** 의 차원을 **주성분 분석(PCA)** 으로 축소하여 사용하고, 나이브 베이즈를 학습하기 위해 **신용카드 채무 불이행 예측 데이터셋** 을 활용합니다. 두 모델의 근본적인 철학과 장단점을 이해하며 분류 문제에 대한 시야를 넓혀보겠습니다.


-----

### 1. 서포트 벡터 머신 (Support Vector Machine, SVM)

SVM의 핵심 아이디어는 매우 직관적입니다. 두 클래스의 데이터를 가장 잘 나누는 선(또는 초평면)은 어떤 모습일까요?

바로 양쪽 클래스로부터 가장 멀리 떨어져 있는 선, 즉 **마진(Margin)을 최대화**하는 선입니다.

#### 🧠 개념 이해하기

  * **결정 경계(Decision Boundary)와 초평면(Hyperplane)**: n차원 공간에서 데이터를 나누는 n-1차원의 부분 공간입니다. 2차원에서는 '선', 3차원에서는 '평면'이 됩니다.
  * **서포트 벡터(Support Vectors)**: 결정 경계를 정의하는 데 가장 중요한, 경계선에 가장 가까이 위치한 데이터 포인트들입니다. 이름 그대로 결정 경계를 '지지(support)'하는 벡터들이며, 이들을 제외한 나머지 데이터는 결정 경계에 영향을 주지 않습니다.
  
    <img src="https://cdn.imweb.me/upload/S202101041a4e45576971e/9d10ef4832fcc.png" width="500">
  * **마진(Margin)**: 결정 경계와 서포트 벡터 사이의 거리를 의미합니다. SVM은 이 마진을 최대화하는 것을 목표로 합니다.

    <img src="https://cdn.imweb.me/upload/S202101041a4e45576971e/aec8f4d0c83dc.png" width="500">

  * **하드 마진(Hard Margin) vs 소프트 마진(Soft Margin)**:
      * **하드 마진**: 단 하나의 오차도 허용하지 않고 모든 데이터를 완벽하게 분리하는 방식입니다. 데이터가 선형적으로 완벽히 분리될 때만 가능하며, 이상치(Outlier)에 매우 민감합니다.
  
      * **소프트 마진**: 약간의 오분류를 허용하더라도 마진을 최대한 넓게 유지하려는 방식입니다. `C` 파라미터를 통해 이 허용 수준을 조절합니다.
  
          * `C` 값이 **크면 클수록**: 오분류에 대한 페널티가 커져 마진이 좁아지고, 모델이 훈련 데이터에 과적합(Overfitting)될 경향이 있습니다. (하드 마진에 가까워짐)
  
          * `C` 값이 **작으면 작을수록**: 오분류에 대한 페널티가 작아져 마진이 넓어지고, 모델이 단순해져 과소적합(Underfitting)될 수 있습니다.
    <p style="text-align: justify;"><img src="https://cdn.imweb.me/upload/S202101041a4e45576971e/3a68a343531f4.png" class="fr-fil fr-dii _img_light_gallery cursor_pointer" data-files="[object Object]" style="width: 425px; height: 400px;" data-src="https://cdn.imweb.me/upload/S202101041a4e45576971e/3a68a343531f4.png"><img src="https://cdn.imweb.me/upload/S202101041a4e45576971e/8617cec616b98.png" class="fr-fil fr-dib _img_light_gallery cursor_pointer" data-files="[object Object]" style="width: 425px; height: 400px;" data-src="https://cdn.imweb.me/upload/S202101041a4e45576971e/8617cec616b98.png"></p>

#### 1.1 선형 SVM (Linear SVM)

데이터가 선형적으로 구분 가능할 때 사용하는 가장 기본적인 SVM입니다.

##### 💻 코드로 알아보기

`scikit-learn`의 `make_blobs`를 사용해 선형 분리가 가능한 가상 데이터를 만들고 `LinearSVC`를 적용해 보겠습니다.

In [1]:
import numpy as np
import plotly.express as px
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# 1. 선형 분리 가능한 데이터 생성
X, y = make_blobs(n_samples=100, centers=2, random_state=42, cluster_std=1.2)
X_scaled = StandardScaler().fit_transform(X) # SVM은 거리에 민감하므로 스케일링이 중요

In [2]:
# 2. LinearSVC 모델 학습
# C=1: 적절한 수준의 규제
model_c1 = LinearSVC(C=1.0, random_state=42, dual=True) # dual=True로 설정해야 수렴 보장
model_c1.fit(X_scaled, y)
pred_c1 = model_c1.predict(X_scaled)
print(f"Accuracy (C=1.0): {accuracy_score(y, pred_c1):.4f}")

# C=100: 규제를 약하게 (오분류 페널티 크게)
model_c100 = LinearSVC(C=100.0, random_state=42, dual=True, max_iter=2000)
model_c100.fit(X_scaled, y)
pred_c100 = model_c100.predict(X_scaled)
print(f"Accuracy (C=100.0): {accuracy_score(y, pred_c100):.4f}")

Accuracy (C=1.0): 1.0000
Accuracy (C=100.0): 1.0000


In [3]:
# 3. 결정 경계 시각화를 위한 함수
def plot_decision_boundary(X, y, model, title):
    # 데이터 포인트 시각화
    df = pd.DataFrame(dict(x1=X[:,0], x2=X[:,1], y=y))
    fig = px.scatter(df, x='x1', y='x2', color='y',
                     title=title,
                     color_continuous_scale=px.colors.sequential.Viridis)

    # 결정 경계 그리기
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    fig.add_contour(x=xx[0], y=yy[:, 0], z=Z,
                    showscale=False,
                    colorscale='gray',
                    opacity=0.3)
    fig.update_layout(width=700, height=500)
    fig.show()

In [4]:

# 4. 시각화 실행
plot_decision_boundary(X_scaled, y, model_c1, "Linear SVM Decision Boundary (C=1.0)")

#### 1.2 커널 SVM (Kernel SVM)

만약 데이터가 아래 그림처럼 선형으로 분리할 수 없다면 어떻게 할까요?


#### 🧠 개념 이해하기

  * **커널 트릭(Kernel Trick)**: SVM의 가장 강력한 기능입니다. 데이터를 직접 더 높은 차원으로 변환하지 않고도, 고차원 공간에서 연산한 것과 같은 효과를 내는 '트릭'입니다.
  
      * **예시**: 1차원의 비선형 데이터를 2차원으로 매핑하여 선형 분리가 가능하게 만드는 것을 상상해 보세요. 커널 트릭은 이 복잡한 변환 과정을 간단한 커널 함수 계산으로 대체합니다.
    <p style="text-align: justify;"><span style="font-size: 18px;"><img src="https://cdn.imweb.me/upload/S202101041a4e45576971e/917cec0b9e232.png" class="fr-fil fr-dii _img_light_gallery cursor_pointer" data-files="[object Object]" style="width: 425px; height: 400px;" data-src="https://cdn.imweb.me/upload/S202101041a4e45576971e/917cec0b9e232.png"><img src="https://cdn.imweb.me/upload/S202101041a4e45576971e/1c67b7192cb37.png" class="fr-fil fr-dib _img_light_gallery cursor_pointer" data-files="[object Object]" style="width: 425px; height: 400px;" data-src="https://cdn.imweb.me/upload/S202101041a4e45576971e/1c67b7192cb37.png"></span></p>

  * **RBF (Radial Basis Function) 커널**: 가장 널리 쓰이는 커널 중 하나로, 모든 점과의 거리를 계산하여 가우시안 분포 형태로 변환합니다. 비선형적인 결정 경계를 만들 수 있습니다.
      * `gamma` 파라미터: RBF 커널의 모양을 결정하며, 하나의 데이터 포인트가 영향을 미치는 거리를 조절합니다.
          * `gamma` 값이 **크면 클수록**: 영향력이 작아져(local) 결정 경계가 매우 구불구불해지고, 모델이 훈련 데이터에 과적합될 경향이 있습니다.
          * `gamma` 값이 **작으면 작을수록**: 영향력이 커져(global) 결정 경계가 부드러워지고, 모델이 단순해집니다.
   
    <img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*6nR_sMAK1OECelJd-TF_4Q.png">
    <br/>
    <img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*dE_SI6I0EBFDJFuY6TDLfg.png">
 
     <img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*uBWPRvaeFsnCVXUTubi4jA.png" style="display: inline-block; width: 45%;">
     <img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*kO_kAQ32-qmT-iljdZdkrQ.png" style="display: inline-block; width: 49%;">



#### 1.3 주성분 분석 (Principal Component Analysis, PCA) + SVM

Fashion-MNIST 데이터셋은 28x28=784개의 픽셀, 즉 784차원의 데이터입니다. 

이처럼 고차원 데이터는 SVM 학습에 많은 시간과 계산 비용을 요구합니다.

 **PCA** 는 이러한 **차원의 저주(Curse of Dimensionality)** 를 해결하는 대표적인 차원 축소 기법입니다.

#### 🧠 개념 이해하기

  * **PCA의 목표**: 데이터의 분산(Variance)을 가장 잘 보존하는 새로운 축(주성분, Principal Components)을 찾는 것입니다. 즉, 정보 손실을 최소화하면서 데이터를 저차원으로 압축합니다.
  * **Explained Variance Ratio**: 각 주성분이 전체 데이터 분산의 몇 %를 설명하는지를 나타내는 지표입니다. 이 값의 누적합을 보고 몇 개의 주성분을 사용할지 결정할 수 있습니다. 
  
    (e.g., "상위 50개 주성분이 전체 분산의 95%를 설명하므로, 784차원 대신 50차원만 사용하자\!")

##### 💻 코드로 알아보기: Fashion-MNIST에 PCA + SVM 적용하기

784차원의 Fashion-MNIST 데이터를 PCA로 2차원으로 축소하여 커널 SVM이 어떻게 클래스를 구분하는지 시각화하고, 파이프라인으로 전체 과정을 묶어보겠습니다.

In [3]:
import numpy as np

In [7]:
from sklearn.preprocessing import StandardScaler

In [1]:
import plotly.graph_objects as go
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.svm import SVC

# 1. 데이터 로드 및 전처리
# (시간이 다소 소요될 수 있습니다)
fashion_mnist = fetch_openml('Fashion-MNIST', version=1, as_frame=False)
X, y = fashion_mnist.data, fashion_mnist.target
X.shape, y.shape

((70000, 784), (70000,))

In [4]:
# 데이터가 크므로 10,000개만 샘플링하여 사용
np.random.seed(42)
sample_idx = np.random.choice(X.shape[0], 10000, replace=False)
X_sample, y_sample = X[sample_idx], y[sample_idx]
X_sample.shape, y_sample.shape

((10000, 784), (10000,))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.3, random_state=42)

In [14]:
# 2. 전처리 및 모델링 파이프라인 구축
# StandardScaler -> PCA(2차원으로 축소) -> SVC(RBF 커널)
pca_svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2, random_state=42)),
    ('svm', SVC(kernel='rbf', random_state=42))
])

In [15]:
# 3. 파이프라인 학습
pca_svm_pipeline.fit(X_train, y_train)
print("PCA+SVM Pipeline 학습 완료.")

PCA+SVM Pipeline 학습 완료.


In [16]:
# 4. 테스트 데이터 평가
accuracy = pca_svm_pipeline.score(X_test, y_test)
print(f"PCA(2D) + SVM Accuracy: {accuracy:.4f}")

PCA(2D) + SVM Accuracy: 0.5333


In [ ]:
# 5. 2차원으로 축소된 데이터 및 결정 경계 시각화
X_train_pca = pca_svm_pipeline.named_steps['scaler'].transform(X_train)
X_train_pca = pca_svm_pipeline.named_steps['pca'].transform(X_train_pca)
model = pca_svm_pipeline.named_steps['svm']

# 클래스 이름 정의
class_names = fashion_mnist.categories

# 결정 경계 계산
x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.astype(int).reshape(xx.shape)

In [18]:
from plotly.subplots import make_subplots
from plotly import express as px
from plotly import graph_objects as go

In [19]:

# 데이터 포인트 시각화
fig = px.scatter(x=X_train_pca[:, 0], y=X_train_pca[:, 1], color=y_train,
                 labels={'color': 'Class'},
                 title='Fashion-MNIST PCA(2D) + SVM Decision Boundary')

# 결정 경계 추가
fig.add_trace(
    go.Contour(x=xx[0], y=yy[:, 0], z=Z,
               showscale=False,
               colorscale='Greys',
               opacity=0.3,
               name='Decision Boundary',
               hoverinfo='none'
              )
)

fig.update_layout(width=800, height=600)
fig.show()

2차원만으로도 어느 정도 클래스들이 군집을 이루고, SVM이 복잡한 결정 경계를 만들어내는 것을 볼 수 있습니다. 실제 예측 성능을 높이려면 더 많은 주성분을 사용해야 합니다.

##### ✏️ 연습문제 1: 최적의 `n_components` 찾기

PCA를 사용할 때, 정보 손실을 최소화하면서 차원을 얼마나 줄일지 결정하는 것이 중요합니다. 

`PCA`의 `explained_variance_ratio_` 속성을 이용하여 **분산의 95%를 설명하기 위해 몇 개의 주성분이 필요한지** 계산하고, 결과를 시각화해보세요.

In [22]:
# 연습문제 1 코드
# StandardScaler로 전체 샘플 데이터(X_sample) 스케일링
scaler = StandardScaler()
X_sample_scaled = scaler.fit_transform(X_sample)

# PCA 객체 생성 (모든 주성분을 확인하기 위해 n_components를 설정하지 않음)
pca = PCA()
pca.fit(X_sample_scaled)
pca.explained_variance_ratio_


array([2.22329086e-01, 1.45905692e-01, 5.52923026e-02, 5.02621708e-02,
       3.98715789e-02, 2.90536175e-02, 2.68847022e-02, 2.32606134e-02,
       1.73542157e-02, 1.31662109e-02, 1.17362538e-02, 9.67750533e-03,
       9.36022608e-03, 8.52439253e-03, 7.48381585e-03, 7.23021434e-03,
       6.65257654e-03, 6.51250641e-03, 6.16584784e-03, 6.05992955e-03,
       5.30489309e-03, 5.04114453e-03, 4.81516157e-03, 4.71011355e-03,
       4.62437803e-03, 4.52173220e-03, 4.19081366e-03, 4.00674834e-03,
       3.91437249e-03, 3.79330978e-03, 3.69101080e-03, 3.62279178e-03,
       3.50307845e-03, 3.37533971e-03, 3.31830565e-03, 3.17969378e-03,
       3.14943647e-03, 3.05425281e-03, 2.92631786e-03, 2.89829443e-03,
       2.78975166e-03, 2.74128709e-03, 2.68548163e-03, 2.58565731e-03,
       2.55270646e-03, 2.45532099e-03, 2.37330558e-03, 2.29254029e-03,
       2.23217603e-03, 2.15974493e-03, 2.14695445e-03, 2.06092185e-03,
       2.05187061e-03, 2.01722674e-03, 1.97689960e-03, 1.94651163e-03,
      

In [ ]:
# 누적 설명 분산 계산
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
cumulative_variance


In [23]:
# 95% 분산을 설명하는 지점 찾기
# 여기에 코드를 작성하여 n_components_95를 구하세요.
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"95%의 분산을 설명하는 주성분 개수: {n_components_95}")


# 누적 설명 분산 시각화
fig = px.area(
    x=range(1, cumulative_variance.shape[0] + 1),
    y=cumulative_variance,
    labels={"x": "주성분 개수", "y": "누적 설명 분산 비율"},
    title="PCA 누적 설명 분산"
)
fig.add_shape(type="line", x0=0, y0=0.95, x1=n_components_95, y1=0.95, line=dict(color="Red", dash="dash"))
fig.add_shape(type="line", x0=n_components_95, y0=0, x1=n_components_95, y1=0.95, line=dict(color="Red", dash="dash"))
fig.show()

95%의 분산을 설명하는 주성분 개수: 243


##### ✏️ 연습문제 2: `GridSearchCV`로 최적의 SVM 파라미터 찾기

`Pipeline`과 `GridSearchCV`를 함께 사용하면 전처리 파라미터와 모델 하이퍼파라미터를 동시에 최적화할 수 있습니다.

 위에서 구한 `n_components_95`를 PCA에 적용하고, SVM의 `C`와 `gamma`에 대한 최적값을 찾아보세요.

In [ ]:
from sklearn.model_selection import GridSearchCV

# 연습문제 2 코드
# 1. 파이프라인 정의
# n_components는 연습문제 1에서 구한 값으로 설정
pipe_for_grid = Pipeline([
    ('scaler', ??),
    ('pca', ??), # n_components_95 값 사용
    ('svm', ??),
])

# 2. 탐색할 하이퍼파라미터 그리드 정의
# 파이프라인의 각 단계 이름과 파라미터 이름을 '__'로 연결
param_grid = {
    'svm__C': ??
    'svm__gamma': ??
}

# 3. GridSearchCV 객체 생성 및 학습
# cv=3 (3-fold cross-validation)
# n_jobs=-1 (사용 가능한 모든 CPU 코어 사용)
grid_search = GridSearchCV(pipe_for_grid, param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 4. 최적 파라미터 및 성능 출력


-----

### 2\. 나이브 베이즈 분류기 (Naive Bayes Classifier)

나이브 베이즈는 데이터의 확률적 속성을 기반으로 분류하는 알고리즘입니다. '이런 특성들이 주어졌을 때, 이 데이터가 특정 클래스에 속할 확률은 얼마일까?'라는 질문에 답하는 방식입니다.

#### 🧠 개념 이해하기

  * **베이즈 정리 (Bayes' Theorem)**: 나이브 베이즈의 근간이 되는 정리입니다.

    $$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

    이를 분류 문제에 맞게 다시 쓰면 다음과 같습니다.

    $$P(\text{클래스}|\text{데이터}) = \frac{P(\text{데이터}|\text{클래스}) \cdot P(\text{클래스})}{P(\text{데이터})}$$

      * $P(\text{클래스}|\text{데이터})$: **사후 확률 (Posterior)**. 우리가 구하고 싶은 값. 데이터가 주어졌을 때, 특정 클래스일 확률.
      * $P(\text{데이터}|\text{클래스})$: **가능도 (Likelihood)**. 특정 클래스라는 가정 하에, 이런 데이터가 관측될 확률.
      * $P(\text{클래스})$: **사전 확률 (Prior)**. 데이터와 무관하게, 특정 클래스가 나타날 고유한 확률.
      * $P(\text{데이터})$: **증거 (Evidence)**. 데이터가 나타날 확률. (모든 클래스에서 동일하므로 보통 무시됨) 

  * **"순진한(Naive)" 가정**: 나이브 베이즈의 핵심 가정입니다. **'모든 특성(feature)은 서로 독립적이다'** 라고 가정합니다.

      * **예시**: 신용카드 연체 예측에서 '연봉'과 '나이'는 서로 아무 관련이 없다고 가정하는 것과 같습니다. 현실에서는 그렇지 않지만, 이 가정이 계산을 매우 단순하고 빠르게 만들어 줍니다.

  * **가우시안 나이브 베이즈 (Gaussian Naive Bayes)**: 연속적인(continuous) 특성을 다룰 때 사용됩니다. 각 클래스에 속한 특성들의 분포가 **가우시안(정규) 분포**를 따른다고 가정하고, 각 분포의 평균과 표준편차를 학습하여 가능도를 계산합니다.

#### 💻 코드로 알아보기: 신용카드 채무 불이행 예측

UCI의 신용카드 채무 불이행 데이터셋을 사용하여 `GaussianNB` 모델을 학습시켜 보겠습니다.

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score

# 1. 데이터 로드 및 전처리
# UCI 데이터셋 로드
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls'
# openpyxl 라이브러리 필요: pip install openpyxl
df_credit = pd.read_excel(url, header=1)


In [13]:
# 컬럼명 정리
df_credit.rename(columns={'default payment next month': 'default'}, inplace=True)
df_credit.drop('ID', axis=1, inplace=True)

In [14]:
# 특성(X)과 타겟(y) 분리
X = df_credit.drop('default', axis=1)
y = df_credit['default']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [15]:
# 2. 전처리 파이프라인 구축
# 수치형, 범주형 변수 식별
numeric_features = ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
# PAY_0, PAY_2~6은 범주형으로 취급하는 것이 좋으나, 여기서는 간소화를 위해 수치형으로 간주
# SEX, EDUCATION, MARRIAGE는 범주형
categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE']

# ColumnTransformer 정의
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ], remainder='passthrough') # 나머지 컬럼(PAY_0 등)은 그대로 통과

In [16]:
# 3. 모델링 파이프라인 구축
gnb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GaussianNB())
])

# 4. 학습 및 평가
gnb_pipeline.fit(X_train, y_train)
y_pred = gnb_pipeline.predict(X_test)
y_pred_proba = gnb_pipeline.predict_proba(X_test)[:, 1]

print("--- Gaussian Naive Bayes Classification Report ---")
print(classification_report(y_test, y_pred))
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")

--- Gaussian Naive Bayes Classification Report ---
              precision    recall  f1-score   support

           0       0.92      0.10      0.18      7009
           1       0.23      0.97      0.38      1991

    accuracy                           0.29      9000
   macro avg       0.58      0.53      0.28      9000
weighted avg       0.77      0.29      0.22      9000

ROC-AUC Score: 0.7253


결과를 보면, 특히 연체(클래스 1)에 대한 재현율(recall)이 상대적으로 낮은 것을 볼 수 있습니다. 이는 나이브 베이즈가 이 데이터의 복잡한 관계를 잘 포착하지 못했음을 시사할 수 있습니다. 하지만 모델 학습 속도가 매우 빠르다는 장점이 있습니다.

##### ✏️ 연습문제 3: 사전 확률(Prior) 확인하기

학습된 `GaussianNB` 모델이 데이터로부터 학습한 각 클래스의 \*\*사전 확률(Prior Probability)\*\*을 확인해보세요. `gnb_pipeline`의 `classifier` 단계에 접근하여 `class_prior_` 속성을 출력하면 됩니다. 이 값은 전체 훈련 데이터의 클래스 비율과 유사할 것입니다.

In [ ]:
# 연습문제 3 코드
# 파이프라인에서 학습된 모델 접근
trained_gnb = gnb_pipeline.named_steps['classifier']

# 사전 확률 출력
# 여기에 코드를 작성하세요.

# 훈련 데이터의 실제 클래스 비율과 비교